<a href="https://colab.research.google.com/github/divya-israni/cadence1b/blob/job_posts_cleaning/JobPostsCleanFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Normalization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
import numpy
import spacy
import re
from collections import Counter

# Try reading the CSV with a different engine and handle bad lines
try:
    df = pd.read_csv('/content/data job posts.csv', engine='python', on_bad_lines='skip')
    display(df.head())
except Exception as e:
    print(f"An error occurred: {e}")
    # If the above fails, try specifying a different delimiter or quote character if known
    # df = pd.read_csv('/content/data job posts.csv', engine='python', on_bad_lines='skip', sep='|', quotechar='"')
    # display(df.head())

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [13]:
print(df.isna().sum())
print(df.columns)

jobpost                0
date                   0
Title                  9
Company                7
AnnouncementCode    6597
Term                4630
Eligibility         5460
Audience            7022
StartDate           3623
Duration            3464
Location              31
JobDescription      1739
JobRequirment       1373
RequiredQual         259
Salary              3737
ApplicationP          44
OpeningDate          629
Deadline              45
Notes               6340
AboutC              2406
Attach              6951
Year                   0
Month                  0
IT                     0
dtype: int64
Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')


In [14]:
df.drop(columns=['AnnouncementCode', 'Audience', 'Notes', 'Attach', 'Term','Eligibility', 'IT','StartDate', 'Duration', 'Deadline', 'OpeningDate', 'ApplicationP', 'AboutC', 'Year', 'Month', 'Salary', 'date'], inplace=True)
df.drop_duplicates(inplace=True)
df = df.dropna(subset=['Title', 'JobDescription', 'RequiredQual'])
df.reset_index(drop=True, inplace=True)`

In [15]:
df.head(10)

,jobpost,Title,Company,Location,JobDescription,JobRequirment,RequiredQual
0,AMERIA Investment Consulting Company\r\nJOB TI...,Chief Financial Officer,AMERIA Investment Consulting Company,"Yerevan, Armenia",AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\r\nindivi..."
1,Caucasus Environmental NGO Network (CENN)\r\nJ...,Country Coordinator,Caucasus Environmental NGO Network (CENN),"Yerevan, Armenia",Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ..."
2,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,BCC Specialist,Manoff Group,"Manila, Philippines",The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci..."
3,"Boutique ""Appollo""\r\nJOB TITLE: Saleswoman\r...",Saleswoman,"Boutique ""Appollo""","Yerevan, Armenia\r\nWORK HOURS: 10:00 - 20:00;...",Saleswoman will sell menswear and accessories.,NaN,"- Candidates should be female, 20-30 years old..."
4,OSI Assistance Foundation - Armenian Branch Of...,Chief Accountant/ Finance Assistant,OSI Assistance Foundation - Armenian Branch Of...,"Yerevan, Armenia",The Armenian Branch Office of the Open Society...,NaN,- University degree in finance/ accounting; \r...
5,American Embassy Yerevan\r\nANNOUNCEMENT NUMBE...,"Program Assistant (INL), FSN-8; FP-6*",American Embassy Yerevan\r\nANNOUNCEMENT NUMBE...,NaN,The incumbent assists in coordinating INL-fund...,NaN,NOTE: All applicants are instructed to\r\naddr...
6,Institute for Sustainable Communities (ISC)\r\...,Chief of Party (COP),Institute for Sustainable Communities (ISC),"Tashkent, Uzbekistan Pending funding.",ISC seeks an experienced and creative Chief of...,NaN,- Masters degree with minimum of seven years o...
7,Food Security Regional Cooperation and Stabili...,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,"Ijevan town, Tavush, Armenia.",Food Security Regional Cooperation and Stabili...,- Assist the Tavush Marz communities and commu...,- Higher Education and/or professional experie...
8,"United Nations Development Programme, Armenia\...",Country Economist (NOB),"United Nations Development Programme, Armenia",NaN,The United Nations Development Programme in Ar...,The incumbent under direct supervision of UNDP...,- Minimum Masters Degree in Economics;\r\n- Mi...
9,"Counterpart International, Inc.\r\nCounterpart...",Driver/ Logistics Assistant,"Counterpart International, Inc.\r\nCounterpart...","Yerevan, Armenia",Driver/ Logistics Assistant is a full-time nat...,- Maintain CHAP/Armenia pool of vehicles and p...,"- Possession of personal vehicle, valid driver..."


In [25]:
df['JobRequirment'] = df['JobRequirment'].fillna('')

In [16]:
df["CleanText"] = (
    df["Title"].astype(str) + " " +
    df["JobDescription"].astype(str) + " " +
    df["JobRequirment"].astype(str) + " " +
    df["RequiredQual"].astype(str)
)

In [17]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', ' ', text)     # remove HTML tags
    text = re.sub(r'[^a-z0-9 ]', ' ', text) # remove special chars
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

for col in ["CleanText"]:
    df[col] = df[col].fillna('').apply(clean_text).apply(lemmatize_text)

In [18]:
df.head()

,jobpost,Title,Company,Location,JobDescription,JobRequirment,RequiredQual,CleanText
0,AMERIA Investment Consulting Company\r\nJOB TI...,Chief Financial Officer,AMERIA Investment Consulting Company,"Yerevan, Armenia",AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\r\nindivi...",chief financial officer ameria investment cons...
1,Caucasus Environmental NGO Network (CENN)\r\nJ...,Country Coordinator,Caucasus Environmental NGO Network (CENN),"Yerevan, Armenia",Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",country coordinator public outreach strengthen...
2,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,BCC Specialist,Manoff Group,"Manila, Philippines",The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",bcc specialist lead local enhancement developm...
3,"Boutique ""Appollo""\r\nJOB TITLE: Saleswoman\r...",Saleswoman,"Boutique ""Appollo""","Yerevan, Armenia\r\nWORK HOURS: 10:00 - 20:00;...",Saleswoman will sell menswear and accessories.,NaN,"- Candidates should be female, 20-30 years old...",saleswoman saleswoman sell menswear accessory ...
4,OSI Assistance Foundation - Armenian Branch Of...,Chief Accountant/ Finance Assistant,OSI Assistance Foundation - Armenian Branch Of...,"Yerevan, Armenia",The Armenian Branch Office of the Open Society...,NaN,- University degree in finance/ accounting; \r...,chief accountant finance assistant armenian br...


In [19]:
df.isnull().sum()

,0
jobpost,0
Title,0
Company,0
Location,12
JobDescription,0
JobRequirment,1042
RequiredQual,0
CleanText,0


In [24]:
# Check for any non-ASCII characters remaining
non_ascii_count = df['CleanText'].apply(lambda x: any(c for c in x if not c.isascii())).sum()
print("Number of rows with non-ASCII characters:", non_ascii_count)

Number of rows with non-ASCII characters: 0


In [20]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [21]:
df["word_count"] = df["CleanText"].apply(lambda x: len(x.split()))
print("Average words per posting:", df["word_count"].mean())
print("Max words in a posting:", df["word_count"].max())
print("Min words in a posting:", df["word_count"].min())

Average words per posting: 120.93145523057599
Max words in a posting: 848
Min words in a posting: 10


In [22]:
all_words = " ".join(df["CleanText"])
word_freq = Counter(all_words.split())
print("Top 20 most common words:", word_freq.most_common(20))

Top 20 most common words: [('work', 10949), ('experience', 10128), ('knowledge', 9789), ('skill', 7960), ('project', 5919), ('ability', 5909), ('language', 5599), ('development', 5127), ('english', 5082), ('good', 4451), ('management', 4339), ('year', 4211), ('team', 4170), ('armenian', 4062), ('excellent', 3947), ('report', 3625), ('software', 3549), ('office', 3499), ('provide', 3447), ('system', 3432)]


In [26]:
df['JobRequirment'].isna().sum()


np.int64(0)

In [27]:
print(df.columns)

Index(['jobpost', 'Title', 'Company', 'Location', 'JobDescription',
       'JobRequirment', 'RequiredQual', 'CleanText', 'word_count'],
      dtype='object')


In [28]:
df['tokens'] = df['CleanText'].map(lambda x: x.split())
df['token_len'] = df['tokens'].map(len)
df[['CleanText', 'tokens', 'token_len']].head()


,CleanText,tokens,token_len
0,chief financial officer ameria investment cons...,"[chief, financial, officer, ameria, investment...",482
1,country coordinator public outreach strengthen...,"[country, coordinator, public, outreach, stren...",85
2,bcc specialist lead local enhancement developm...,"[bcc, specialist, lead, local, enhancement, de...",334
3,saleswoman saleswoman sell menswear accessory ...,"[saleswoman, saleswoman, sell, menswear, acces...",31
4,chief accountant finance assistant armenian br...,"[chief, accountant, finance, assistant, armeni...",79
